In [1]:
import os
import sys
import pandas as pd
from arcgis.features import GeoAccessor, FeatureSet
from arcgis.geometry import Geometry
from arcgis.gis import GIS
from arcgis.network.analysis import find_closest_facilities, generate_origin_destination_cost_matrix
import json
import math

In [4]:
import sys
sys.path.append(os.path.abspath('../../src/geoai_retail'))
from proximity_rest import *
sys.path.append(os.path.abspath('../'))
import config

%load_ext autoreload
%autoreload 2

In [5]:
dest_id = '7a38868f3d754d76947e286346a2b6e9'
dest_fc = r'D:\projects\geoai-retail\data\interim\interim.gdb\stores_423820'
dest_id_fld = 'LOCNUM'
dest_brand_fld = 'CONAME'
origin_id = '67720de48dfc4d0fb98bf661dc383a7a'
origin_fc = r'D:\projects\geoai-retail\data\interim\interim.gdb\blocks'
origin_pt_csv = r'D:\projects\geoai-retail\data\interim\block_points.csv'
origin_pt_fc = r'D:\projects\geoai-retail\data\interim\interim.gdb\block_points'
origin_id_fld = 'GEOID'
dest_count = 4

In [6]:
geoai_gis = GIS(config.agol_url, username=config.agol_user, password=config.agol_pass)
geoai_gis

GIS @ https://geoai.maps.arcgis.com version:7.2

In [7]:
dev_gis = GIS(config.dev_url, username=config.dev_user, password=config.dev_pass)
dev_gis

GIS @ https://notebooks.mapsqa.arcgis.com version:7.2

In [8]:
ent_gis = GIS(config.ent_url, username=config.ent_user, password=config.ent_pass)
ent_gis

GIS @ https://geoai-webgis-ent.bd.esri.com/portal version:7.1

In [9]:
%%time
#origin_df = prep_feature_service_for_nearest(origin_id, origin_id_fld, geoai_gis)
origin_df_poly = GeoAccessor.from_featureclass(origin_pt_fc)
origin_df = prep_sdf_for_nearest(origin_df_poly, origin_id_fld)
print(f'origin feaure count - {len(origin_df.index):,}')

origin feaure count - 363,127
Wall time: 10 s


In [10]:
%%time
# dest_df = prep_feature_service_for_nearest(dest_id, dest_id_fld, geoai_gis)
dest_df = prep_sdf_for_nearest(GeoAccessor.from_featureclass(dest_fc), dest_id_fld)
print(f'destination feature count - {len(dest_df.index):,}')

destination feature count - 167
Wall time: 261 ms


In [11]:
origin_df_orig = origin_df

In [12]:
origin_df = origin_df_orig.iloc[:2500].copy()
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 3 columns):
ID       2500 non-null object
Name     2500 non-null object
SHAPE    2500 non-null geometry
dtypes: geometry(1), object(2)
memory usage: 58.7+ KB


In [13]:
%%time
closest_df = closest_dataframe_from_dataframes(origin_df, origin_id_fld, dest_df, dest_id_fld, ent_gis)
closest_df.info()

NameError: name 'closest_dataframe_from_dataframes' is not defined

In [20]:
closest_df.sample(20)

,origin_id,destination_id_01,proximity_kilometers_01,proximity_minutes_01,destination_id_02,proximity_kilometers_02,proximity_minutes_02,destination_id_03,proximity_kilometers_03,proximity_minutes_03,destination_id_04,proximity_kilometers_04,proximity_minutes_04
612,060490002001612,529260200,302.106782,243.698249,997233473,306.159934,250.323302,605349117,308.557254,258.085542,355625807,309.381132,249.436204
1296,060930001001478,529260200,283.965477,233.025612,997233473,288.018629,239.650665,605349117,290.415949,247.412905,355625807,291.239827,238.763567
493,060490002001493,529260200,301.143459,247.825213,997233473,305.196611,254.450266,605349117,307.593932,262.212506,355625807,308.417809,253.563168
600,060490002001600,529260200,307.423788,240.964741,997233473,311.476939,247.589794,605349117,313.874260,255.352033,355625807,314.698138,246.702696
936,060930001001118,529260200,260.011079,182.973141,997233473,264.064230,189.598194,605349117,266.461551,197.360434,355625807,267.285429,188.711096
856,060930001001038,529260200,253.654957,176.230867,997233473,257.708109,182.855920,605349117,260.105429,190.618160,355625807,260.929307,181.968822
420,060490002001420,529260200,280.824634,197.455642,997233473,284.877786,204.080695,605349117,287.275107,211.842934,355625807,288.098984,203.193597
1656,060930002001341,529260200,275.206737,188.203290,997233473,279.259888,194.828343,605349117,281.657209,202.590582,355625807,282.481087,193.941245
821,060930001001003,529260200,256.516351,178.426428,997233473,260.569503,185.051481,605349117,262.966824,192.813720,355625807,263.790702,184.164383
2133,060930003001036,529260200,292.655411,225.336647,997233473,296.708562,231.961700,605349117,299.105883,239.723939,355625807,299.929761,231.074602


In [21]:
closest_df.to_csv('../data/interim/closest_sample.csv')

In [10]:
origin_poly_fc = GeoAccessor.from_featureclass(origin_fc)
origin_poly_fc.head()

,OBJECTID,GEOID,SUFFIX,NAME,ALAND,AWATER,INTPTLAT,INTPTLON,SHAPE
0,1,060490002001000,None,BLOCK 1000,3147342.0,642.0,+41.9920259,-121.2597021,"{""rings"": [[[-121.28641361999996, 41.997446366..."
1,2,060490002001001,None,BLOCK 1001,10152.0,0.0,+41.9876103,-121.2707465,"{""rings"": [[[-121.27056461599994, 41.988320367..."
2,3,060490002001002,None,BLOCK 1002,5165023.0,91700.0,+41.9705345,-121.2476771,"{""rings"": [[[-121.27050961599997, 41.987279367..."
3,4,060490002001003,None,BLOCK 1003,143674.0,0.0,+41.9517399,-121.2521204,"{""rings"": [[[-121.25547460699994, 41.950711365..."
4,5,060490002001004,None,BLOCK 1004,34144.0,0.0,+41.9534475,-121.2498147,"{""rings"": [[[-121.25137560699994, 41.954012366..."


In [22]:
in_features = origin_fc
gis = ent_gis

In [67]:
from arcgis.features import GeoAccessor, FeatureLayer
import pandas as pd


def get_dataframe(in_features, gis=None):
    """
    Get a spatially enabled dataframe from the input features provided.
    :param in_features: Spatially Enabled Dataframe | String path to Feature Class | String url to Feature Service | String Web GIS Item ID
        Resource to be evaluated and converted to a Spatially Enabled Dataframe.
    """
    # if already a Spatially Enabled Dataframe, mostly just pass it straight through
    if isinstance(in_features, pd.DataFrame) and in_features.spatial.validate() == True:
        df = in_features
        
    # if a csv previously exported from a Spatially Enabled Dataframe, get it in
    if isinstance(in_features, str) and os.path.exists(in_features) and in_features.endswith('.csv'):
        df = pd.read_csv(in_features)
        df['SHAPE'] = df['SHAPE'].apply(lambda geom: Geometry(eval(geom)))
        
        # this almost always is the index written to the csv, so taking care of this
        if df.columns[0] == 'Unnamed: 0':
            df = df.set_index('Unnamed: 0')
            del(df.index.name)
            
    # create a Spatially Enabled Dataframe from the direct url to the Feature Service including the index of the layer in the url
    elif isinstance(in_features, str) and in_features.startswith('http'):
        if gis is not None:
            df = FeatureLayer(in_features, gis).query(out_sr=4326, as_df=True)
        else:
            df = FeatureLayer(in_features).query(out_sr=4326, as_df=True)
    
    # create a Spatially Enabled Dataframe from a Web GIS Item ID
    elif isinstance(in_features, str) and len(in_features) == 32:
        if gis is None:
            raise Exception('If using an Item ID, you MUST also provide the GIS of where to find this resource.')
        itm = gis.content.get(in_features)
        df = itm.layers[0].query(out_sr=4326, as_df=True)
    
    # create a Spatially Enabled Dataframe from a local feature class
    elif isinstance(in_features, str):
        df = GeoAccessor.from_featureclass(in_features)

    # ensure the universal spatial column is correctly being recognized
    df.spatial.set_geometry('SHAPE')
    
    # index the spatial data for faster subsequnet analyses
    df.spatial.sindex()
            
    return df

In [68]:
df = get_dataframe(dest_fc)
df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,SHAPE
0,1,106137557,TRACTOR STORE,W 11TH AVE,EUGENE,OR,Oregon,97402,9103,42382005,...,3.0,,,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -123.19779999999997, ""y"": 44.04860000000..."
1,2,106094592,SITE ONE LANDSCAPE SUPPLY,WALLIS ST,EUGENE,OR,Oregon,97402,2920,42382017,...,5.0,,,4,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -123.15439999999995, ""y"": 44.04680000000..."
2,3,608538914,MODERN MACHINERY CO,CLOUDBURST ST,EUGENE,OR,Oregon,97402,5432,42382014,...,20.0,,,6,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -123.17519999999996, ""y"": 44.06240000000..."
3,4,000820159,PIERCE FITTINGS,GARFIELD ST,EUGENE,OR,Oregon,97402,7600,42382016,...,50.0,,,7,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -123.12389999999999, ""y"": 44.05980000000..."
4,5,608540191,GROWING SOLUTIONS INC,W 2ND AVE,EUGENE,OR,Oregon,97402,4387,42382005,...,7.0,,,4,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -123.11819999999994, ""y"": 44.05670000000..."


In [61]:
df[:1000].copy().spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))